<a href="https://colab.research.google.com/github/pavelmajek/fcc-machine-learning-with-python/blob/main/linear_regression_health_costs_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# formatting the data
data_formatting = pd.read_csv("insurance.csv")
data_formatting["sex"] = data_formatting["sex"].replace({'female': 0, 'male': 1})
data_formatting["smoker"] = data_formatting["smoker"].replace({'no': 0, 'yes': 1})
print(data_formatting["region"].value_counts())
data_formatting["region"] = data_formatting["region"].replace({'southeast': 0, 'southwest': 1, 'northwest': 2, 'northeast': 3})
data_formatting

In [ ]:
# splitting the datasets
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(data_formatting, test_size=0.2, random_state=34)
pop_train = train_dataset.pop("expenses")
train_labels = pd.DataFrame(pop_train)
pop_test = test_dataset.pop("expenses")
test_labels = pd.DataFrame(pop_test)

In [ ]:
# normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

numerical_cols = ['age', 'bmi', 'children']
train_numerical_data = train_dataset[numerical_cols]
test_numerical_data = test_dataset[numerical_cols]

train_normalized_data = scaler.fit_transform(train_numerical_data)
test_normalized_data = scaler.fit_transform(test_numerical_data)

train_dataset[['age', 'bmi', 'children']] = train_normalized_data
test_dataset[['age', 'bmi', 'children']] = test_normalized_data

In [ ]:
# creating the model

model = tf.keras.models.Sequential()
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=1))

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error', 'mean_squared_error'])

history = model.fit(train_dataset, train_labels, epochs=150, validation_data=(test_dataset, test_labels), verbose=1)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
